Input Pipeline and Training

In [ ]:
#!pip install transformers

In [21]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

#paths = [str(x) for x in Path(".").glob("**/*.txt")]
paths = [str(x) for x in Path("/mnt/c/Users/iotfs/OneDrive - Universiti Malaya/dataset/Dataset_csv/unsw/nfstream/corpus/").glob("**/*.csv")]


# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [ ]:
!mkdir models
tokenizer.save_model("models")

In [2]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./models/vocab.json",
    "./models/merges.txt",
)

In [3]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [4]:
seq="sp:4856 dp:49152 ptcl:6 ipv:4 vln:0 tnnl:0 bi_dur:34 bi_pkt:14 bi_byte:4544 s2d_dur:32 s2d:7 s2d_byte:555 d2s_dur:33 d2s:7 d2s_byte:3989 bi_min_ps:52 bi_mean_ps:324.57 bi_std_ps:516.42 bi_max_ps:1500 s2d_min_ps:52 s2d_mean_ps:79.29 s2d_std_ps:68.73 s2d_max_ps:235 d2s_min_ps:52 d2s_mean_ps:569.86 d2s_std_ps:657.82 d2s_max_ps:1500 bi_min_pi_ms:0 bi_mean_pi_ms:2.62 bi_std_pi_ms:2.26 bi_max_pi_ms:6 s2d_min_pi_ms:1 s2d_mean_pi_ms:5.33 s2d_std_pi_ms:5.5 s2d_max_pi_ms:16 d2s_min_pi_ms:0 d2s_mean_pi_ms:5.5 d2s_std_pi_ms:6.06 d2s_max_pi_ms:16 bi_syn:2 bi_cwr:0 bi_ece:0 bi_urg:0 bi_ack:13 bi_psh:3 bi_rst:0 bi_fin:2 s2d_syn:1 s2d_cwr:0 s2d_ece:0 s2d_urg:0 s2d_ack:6 s2d_psh:1 s2d_rst:0 s2d_fin:1 d2s_syn:1 d2s_cwr:0 d2s_ece:0 d2s_urg:0 d2s_ack:7 d2s_psh:2 d2s_rst:0 d2s_fin:1 app_name:HTTP app_cat:Web req_server_name:192.168.1.223 client_fingerprint:nan server_fingerprint:nan content_type:text/xml"
tokenizer.encode(seq)

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode(seq).tokens

In [5]:
#!nvidia-smi
import torch
torch.cuda.is_available()

False

In [6]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [7]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./models", max_len=512)

In [8]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [9]:
model.num_parameters()

83504416

build training dataset

In [10]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/mnt/c/Users/iotfs/OneDrive - Universiti Malaya/dataset/Dataset_csv/unsw/nfstream/corpus/unsw_corpus7.csv",
    block_size=128,
)

/home/pidot/transformer/transformer-env/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 1min 7s, sys: 4.54 s, total: 1min 12s
Wall time: 11.7 s


In [ ]:
%%time

from datasets import load_dataset
# data_files={"train": "./data/unsw_corpus_*.csv", "test": "./data/unsw_corpus_2.csv"}
data_files="./data/unsw_corpus_1.csv"
# data_features = Features({'text': datasets.Value('string')})
dataset = load_dataset('text', data_files=data_files, split="train")


In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [13]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/pidot/transformer/transformer-env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 30121
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 471


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 20h 44min 16s, sys: 22min 26s, total: 21h 6min 43s
Wall time: 5h 38min 8s


TrainOutput(global_step=471, training_loss=1.1944291100633626, metrics={'train_runtime': 20278.6815, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.023, 'total_flos': 998715494719488.0, 'train_loss': 1.1944291100633626, 'epoch': 1.0})

In [14]:
trainer.save_model("./models")

Saving model checkpoint to ./models
Configuration saved in ./models/config.json
Model weights saved in ./models/pytorch_model.bin


: 